In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from bioio import BioImage
import bioio_ome_zarr
import matplotlib.pyplot as plt
from skimage import exposure
from skimage.transform import resize

In [ ]:
fused_path = "/mnt/gdrive/ThNe/development_retina/fused_images_n5/"

In [ ]:
p21 = "P15/M/R/retina_Age_P21_Sex_F_Side_L_Animal_1.zarr"
p15 = "P15/M/R/retina_Age_P15_Sex_M_Side_R_Animal_2_c_12.zarr"

In [ ]:
img_path = fused_path + p21
img = BioImage(img_path, reader=bioio_ome_zarr.Reader)

In [ ]:
print(f"Shape: {img.shape}")
print(f"Dimensions: {img.dims}")
print(f"Metadata keys: {list(img.metadata.keys())}")

# Get image data (Dask or NumPy)
data = img.data
del img


In [ ]:
del img


In [ ]:
if hasattr(data, 'compute'):
    data = data.compute()  # Convert Dask array to NumPy if it's a lazy-loaded array

# Size in GB
size_in_GB = data.nbytes / (1024 ** 3)  # Convert bytes to gigabytes
print(f"Data Size: {size_in_GB:.3f} GB")

# Get pixel type (dtype)
pixel_type = data.dtype
print(f"Pixel Type: {pixel_type}")

In [ ]:
# downsampled_img = resize(img.data, (img.shape[-4], img.shape[-5], img.shape[-3], img.shape[-2] // 2, img.shape[-1] // 2), mode='reflect')


Blood Vessels

In [ ]:
z_list = range(3, 30, 3)
for z in z_list:
    tile = data[0, 0, z,1000:2000, 5000:7000]  # t=0, c=1, z=z

    if hasattr(tile, 'compute'):
        tile = tile.compute()
    tile_clahe = exposure.equalize_adapthist(tile, clip_limit=0.3)

    # Show the tile
    plt.figure(figsize=(6, 6))
    plt.imshow(tile_clahe, cmap='gray')
    plt.title(f"Z-slice {z}")
    plt.axis('off')
    plt.show()

Microglia

In [ ]:
mg_tile = data[0, 0, 25, 3000:4000, 1000:2000]

In [ ]:
z_list = range(22, 26, 1)
for z in z_list:
    tile = data[0, 0, z, 3000:4000, 1000:2000]  # t=0, c=1, z=z

    if hasattr(tile, 'compute'):
        tile = tile.compute()
    tile_clahe = exposure.equalize_adapthist(tile, clip_limit=0.01)

    # Show the tile
    plt.figure(figsize=(6, 6))
    plt.imshow(tile_clahe, cmap='gray')
    plt.title(f"Z-slice {z}")
    plt.axis('off')
    plt.show()

    # # Plot histogram with log scale
    # plt.figure(figsize=(6, 4))
    # plt.hist(tile.ravel(), bins=256, color='gray', log=True)
    # plt.title("Grayscale Intensity Distribution (Log Scale)")
    # plt.xlabel("Pixel Intensity")
    # plt.ylabel("Log Count")
    # plt.grid(True)
    # plt.show()


In [ ]:
from skimage import exposure

tile_eq = exposure.rescale_intensity(tile, in_range='image', out_range='uint8')

plt.figure(figsize=(6, 6))
plt.imshow(tile_eq, cmap='gray')
plt.title("Rescaled Contrast")
plt.axis('off')
plt.show()


In [ ]:
if hasattr(tile, 'compute'):
    tile = tile.compute()
tile_clahe = exposure.equalize_adapthist(tile, clip_limit=0.01)
# Show the tile
plt.figure(figsize=(6, 6))
plt.imshow(tile_clahe, cmap='gray')
plt.title(f"Z-slice {z}")
plt.axis('off')
plt.show()

In [ ]:
del data

In [ ]:
mg_cube = data[0, 1, :, 3000:4000, 1000:2000]

In [ ]:
mg_cube.type

In [ ]:
import numpy as np
from skimage import filters, morphology, measure, exposure
import matplotlib.pyplot as plt
from scipy import ndimage as ndi

# Example usage:
z = 25  # You can change this value to any slice you want to process
labels = process_slice(mg_cube, z)  # Process the image at slice z


In [ ]:
# Extract the slice for the given z
for z in [23,24,25,26]:
    slice_ = mg_cube[z]

    # Step 1: Apply Otsu thresholding (or another thresholding method)
    threshold = filters.threshold_otsu(slice_)  # Automatically finds a threshold
    binary = slice_ > threshold  # Convert the image to binary (True/False)

    # Visualize the thresholded image
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(binary, cmap='gray')  # Show the Z-slice
    plt.title(f"Thresholded Image - Z={z}")

In [ ]:
# Step 2: Enhance contrast for tiny objects
enhanced = exposure.equalize_hist(binary)  # Optionally, try adaptive equalization

# Visualize the enhanced image
plt.subplot(1, 2, 2)
plt.imshow(enhanced, cmap='gray')  # Show the Z-slice
plt.title(f"Enhanced Image - Z={z}")
plt.show()


In [ ]:
"""
    Process the 3D image at a specific z slice.

    Parameters:
    - mg_cube: The 3D image (numpy array or dask array)
    - z: The Z-slice to process (integer)

    Returns:
    - labels: Labeled image after processing
    """



    # Step 3: Convert enhanced to binary (True for foreground, False for background)
    binary_enhanced = enhanced > 0.5  # Threshold the enhanced image to create a binary mask

    # Step 4: Remove small objects (based on the minimum size of objects)
    cleaned = morphology.remove_small_objects(binary_enhanced, min_size=50, connectivity=2)

    # Visualize cleaned image
    plt.figure(figsize=(10, 5))
    plt.imshow(cleaned, cmap='gray')  # Show the Z-slice
    plt.title(f"Cleaned Image - Z={z}")
    plt.show()

    # Step 5: Optional - Remove small holes inside objects
    filled = ndi.binary_fill_holes(cleaned)

    # Visualize filled image
    plt.figure(figsize=(10, 5))
    plt.imshow(filled, cmap='gray')  # Show the Z-slice
    plt.title(f"Filled Image - Z={z}")
    plt.show()

    # Step 6: Apply 3D dilation for enhanced objects (adjust as needed)
    dilated = morphology.dilation(filled, morphology.ball(1))  # Reduce ball size to 1 for a subtle effect

    # Visualize dilated image
    plt.figure(figsize=(10, 5))
    plt.imshow(dilated, cmap='gray')  # Show the Z-slice
    plt.title(f"Dilated Image - Z={z}")
    plt.show()

    # Step 7: Label connected components (this gives you individual objects)
    labels = measure.label(dilated, connectivity=2)

    # Visualize labeled objects
    plt.figure(figsize=(10, 5))
    plt.imshow(labels, cmap='nipy_spectral')  # Show the Z-slice
    plt.title(f"Segmented Objects - Z={z}")
    plt.show()


In [ ]:
# Step 8: Visualize the result
plt.figure(figsize=(10, 10))
plt.subplot(1, 2, 1)
plt.imshow(binary[25], cmap='gray')  # Visualizing only the first slice (Z=0) for simplicity
plt.title("Thresholded Image")

plt.subplot(1, 2, 2)
plt.imshow(labels[26], cmap='nipy_spectral')  # Visualizing only the first slice (Z=0) for labeled objects
plt.title("Segmented Objects")
plt.show()